In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import importlib
import time
import datetime
import json
import threading

import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數
importlib.reload(shioaji_login)

import tools.globals as globals
import tools.get_snap_options as snap
import tools.message_log as message_log
import tools.get_positions as positions

In [2]:
def update_config():
    """
    Threading function.
    每秒更新config相關參數
    
    :global param: get_strike_time
    :return: None
    """

    pre_get_strike_time = None
    pre_simulation_mode = None
    pre_simulation_optionright = None
    pre_simulation_quantity = None
    pre_simulation_action = None
    
    while(True):

        with open('config.json') as f:
            config_data = json.load(f)

            globals.get_strike_time =  datetime.datetime.strptime(config_data['get_strike_time'], '%H:%M:%S').time()

            ### set simulation ###
            if config_data['simulation_mode'].lower() == "false":
                globals.simulation_mode = False
            else:
                globals.simulation_mode = True

            if config_data['simulation_optionright'].lower() == "c":
                globals.simulation_optionright = 'C'
            else:
                globals.simulation_optionright = 'P'

            if config_data['simulation_action'].lower() == "buy":
                globals.simulation_action = 'Buy'
            else:
                globals.simulation_action = 'Sell'

            globals.simulation_quantity = config_data['simulation_quantity']
            
            ### detect ###
            if(pre_get_strike_time != globals.get_strike_time):
                print(f'Get strike time has been set to {globals.get_strike_time}')
                pre_get_strike_time = globals.get_strike_time

            if(pre_simulation_mode != globals.simulation_mode):
                print(f'simulation_mode has been set to {globals.simulation_mode}')
                pre_simulation_mode = globals.simulation_mode

            if(pre_simulation_optionright != globals.simulation_optionright):
                print(f'simulation_optionright has been set to {globals.simulation_optionright}')
                pre_simulation_optionright = globals.simulation_optionright

            if(pre_simulation_action != globals.simulation_action):
                print(f'simulation_action has been set to {globals.simulation_action}')
                pre_simulation_action = globals.simulation_action

            if(pre_simulation_quantity != globals.simulation_quantity):
                print(f'simulation_quantity has been set to {globals.simulation_quantity}')
                pre_simulation_quantity = globals.simulation_quantity

            time.sleep(1)



In [3]:
def update_snap_options():
    """
    Threading function.
    每秒偵測,若現在時間符合get_strike_time, 則找尋當下價平 
    且若simulation_mode為True則下模擬單
    
    :global param: get_strike_time
    :global param: simulation_mode
    :return: None
    """

    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_strike_time):
            snap.get_snap_options()
            snap.get_at_the_money_info()
            print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)

            if globals.simulation_mode:
                option_code = globals.at_the_money_code +  snap.get_option_code(globals.simulation_optionright)
                positions.place_order(globals.simulation_quantity, option_code, globals.simulation_optionright, globals.simulation_action)

        time.sleep(1)

In [4]:
def load_position():
    # 取得Position 
    globals.position = globals.api.list_positions(globals.api.futopt_account)

    if len(globals.position) == 0:
        print("None")

In [5]:
def main():
    """
    Controlled whole program, make app execute.
    
    :global param: api
    """
    # initialize all global variables
    globals.initialize()

    # load_position()

    # log in
    globals.api = shioaji_login.login()
    globals.api.set_order_callback(positions.place_cb)

    # Start update config thread. All config variable will be updated every second.
    update_config_thread = threading.Thread(target = update_config)
    update_config_thread.start()
    time.sleep(1)

    update_snap_options_thread = threading.Thread(target = update_snap_options)
    update_snap_options_thread.start()
    time.sleep(1)

    snap.get_snap_options()
    snap.get_at_the_money_info()
    print("現在價平和之檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
    
    # print(api.Contracts.Options.TX4)



In [6]:
if __name__ == "__main__":
    main() 

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID R124743829
Login status: [FutureAccount(person_id='R124743829', broker_id='F002000', account_id='1697256', signed=True, username='劉庭銘'), Account(account_type=<AccountType.H: 'H'>, person_id='R124743829', broker_id='9A9X', account_id='0150198', username='劉庭銘\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000'), StockAccount(person_id='R124743829', broker_id='9A9X', account_id='0510980', signed=True, username='劉庭銘\u3000\u3000')]
Activating CA at the path C:/sinopac/Sinopac.pfx


Get strike time has been set to 20:16:00
simulation_mode has been set to True
現在價平和之檔位及成交點數和:  TX114950 166.0


In [7]:
# 目前欲實現進度: 在get_strike_time抓到價平後下模擬單，並在達到停損條件後(成交價or強制平倉時間)出掉

現在價平和之檔位及成交點數和:  TX114950 166.0
A deal has been detected.
Deal information: code:TX114950L2, action:Buy, price:88, quantity:1, optionright:C
Delivery month:202212, security type: OPT
***
A position with type=Buy Call, quantity=1, price=88 has been recorded!

***

***
A position with type=Buy Call, quantity=1, price=88 has been added to the track list!

***

